# 04 Manipulación de los Datos:

En este ejercicio vamos a manipular una base de datos de recursos humanos

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Configuración
num_empleados = 50
departamentos = ["Ventas", "TI", "Recursos Humanos", "Finanzas", "Operaciones"]
cargos = ["Analista", "Gerente", "Asistente", "Especialista", "Director"]

# 1. Tabla: Empleados (Maestro)
empleados = []
for i in range(1, num_empleados + 1):
    id_emp = f"E{i:03d}"
    nombre = f"Empleado_{i}"
    depto = random.choice(departamentos)
    cargo = random.choice(cargos)
    salario = random.randint(800, 3500)
    fecha_ingreso = datetime(2018, 1, 1) + timedelta(days=random.randint(0, 2000))
    genero = random.choice(["M", "F"])
    empleados.append([id_emp, nombre, depto, cargo, salario, fecha_ingreso.strftime("%Y-%m-%d"), genero])

df_empleados = pd.DataFrame(empleados, columns=["ID_Empleado", "Nombre", "Departamento", "Cargo", "Salario_Mensual", "Fecha_Ingreso", "Genero"])

# 2. Tabla: Evaluaciones de Desempeño (Hechos - Anual)
# Asumimos la evaluación del último año
evaluaciones = []
for emp in empleados:
    id_emp = emp[0]
    # Score 1 to 5
    score = random.choices([1, 2, 3, 4, 5], weights=[5, 10, 40, 35, 10])[0] # Weighted towards 3 and 4
    fecha_eval = "2023-12-15"
    evaluaciones.append([id_emp, fecha_eval, score])

df_evaluaciones = pd.DataFrame(evaluaciones, columns=["ID_Empleado", "Fecha_Evaluacion", "Puntaje_Desempeno"])

# 3. Tabla: Registro de Asistencia (Hechos - Mensual Resumido)
# Datos de los últimos 3 meses para cada empleado
asistencia = []
meses = ["2024-01", "2024-02", "2024-03"]

for emp in empleados:
    id_emp = emp[0]
    for mes in meses:
        dias_ausente = random.choices([0, 1, 2, 3], weights=[70, 15, 10, 5])[0]
        horas_extra = random.randint(0, 10)
        asistencia.append([id_emp, mes, dias_ausente, horas_extra])

df_asistencia = pd.DataFrame(asistencia, columns=["ID_Empleado", "Mes", "Dias_Ausente", "Horas_Extra"])

# Save to CSV
df_empleados.to_csv("../data/HR_Empleados.csv", index=False)
df_evaluaciones.to_csv("../data/HR_Evaluaciones.csv", index=False)
df_asistencia.to_csv("../data/HR_Asistencia.csv", index=False)

In [ ]:
print("Empleados Head:")
print(df_empleados.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nEvaluaciones Head:")
print(df_evaluaciones.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nAsistencia Head:")
print(df_asistencia.head().to_markdown(index=False, numalign="left", stralign="left"))

Empleados Head:
| ID_Empleado   | Nombre     | Departamento   | Cargo        | Salario_Mensual   | Fecha_Ingreso   | Genero   |
|:--------------|:-----------|:---------------|:-------------|:------------------|:----------------|:---------|
| E001          | Empleado_1 | TI             | Director     | 2231              | 2018-05-04      | F        |
| E002          | Empleado_2 | Finanzas       | Analista     | 3034              | 2019-10-16      | M        |
| E003          | Empleado_3 | TI             | Director     | 2647              | 2019-02-08      | M        |
| E004          | Empleado_4 | Finanzas       | Especialista | 1645              | 2021-12-21      | F        |
| E005          | Empleado_5 | Operaciones    | Director     | 1838              | 2018-07-26      | M        |

Evaluaciones Head:
| ID_Empleado   | Fecha_Evaluacion   | Puntaje_Desempeno   |
|:--------------|:-------------------|:--------------------|
| E001          | 2023-12-15         | 3                  

## 1. El Escenario de Negocio

Eres un analista de sistemas recién contratado en la empresa. El **Director de Recursos Humanos** está preocupado. Siente que el presupuesto de salarios es muy alto y no está seguro de si se está pagando justamente a las personas que más se esfuerzan.

Te ha entregado tres archivos exportados del sistema antiguo de gestión (Legacy System) y te ha dicho:
> *"Tengo estos datos sueltos y no entiendo nada. Necesito saber quiénes son mis empleados estrella y quiénes están ganando mucho dinero pero rindiendo poco. ¡Tráeme un reporte en una hora!"*

## 2. Datos

Descarga los siguientes archivos `.csv` disponibles en el aula virtual:

1.  **`HR_Empleados.csv` (Maestro de Empleados):** Contiene la información personal, cargo y salario.
    * *Llave Principal:* `ID_Empleado`
2.  **`HR_Evaluaciones.csv` (Desempeño):** Contiene el puntaje de la última evaluación anual (Escala 1 a 5).
    * *Llave Foránea:* `ID_Empleado`
3.  **`HR_Asistencia.csv` (Asistencia):** Registro mensual de los últimos 3 meses.
    * *Llave Foránea:* `ID_Empleado`

In [ ]:
import pandas as pd

# -----------------------------
# CARGA DE DATOS
# -----------------------------
ARCHIVOS = {
    "empleados": "HR_Empleados.csv",
    "evaluaciones": "HR_Evaluaciones.csv",
    "asistencia": "HR_Asistencia.csv"
}

df_empleados = pd.read_csv(ARCHIVOS["empleados"])
df_evaluaciones = pd.read_csv(ARCHIVOS["evaluaciones"])
df_asistencia = pd.read_csv(ARCHIVOS["asistencia"])

# -----------------------------
# AGREGADOS DE ASISTENCIA
# -----------------------------
df_asistencia_agg = (
    df_asistencia
    .groupby("ID_Empleado", as_index=False)
    .agg(
        Promedio_Dias_Ausente=("Dias_Ausente", "mean"),
        Promedio_Horas_Extra=("Horas_Extra", "mean")
    )
)

# -----------------------------
# UNIFICACIÓN DE TABLAS
# -----------------------------
df_final = (
    df_empleados
    .merge(
        df_evaluaciones[["ID_Empleado", "Puntaje_Desempeno"]],
        on="ID_Empleado",
        how="left"
    )
    .merge(
        df_asistencia_agg,
        on="ID_Empleado",
        how="left"
    )
)

# -----------------------------
# VISTA RÁPIDA
# -----------------------------
print("\n📊 TABLA UNIFICADA DE RRHH")
print("=" * 35)
print(df_final.head().to_markdown(index=False))


## 3. Instrucciones

### Paso 1: Unificación de Datos (Data Blending)
Los datos están fragmentados (como en la mayoría de sistemas reales). Tu primer trabajo es consolidarlos.
* Abre `HR_Empleados.csv`. Esta será tu tabla principal.
* Usa la función `merge` en Python para traer las siguientes columnas a tu tabla principal, usando el `ID_Empleado` como conector:
    * Trae el `Puntaje_Desempeno` desde el archivo de Evaluaciones.
    * **Reto:** Calcula el *promedio* de `Dias_Ausente` de los 3 meses (desde el archivo de Asistencia) y tráelo a la tabla principal. (Si usas Excel, quizás debas usar Tablas Dinámicas primero para resumir la asistencia por empleado).

### Paso 2: Análisis y Generación de Información (KPIs)
Responde las preguntas del Director filtrando y ordenando los datos:

**A. Identificación de Talentos (Los "Estrellas")**
Filtra la lista para encontrar empleados que cumplan AMBAS condiciones:
1.  `Puntaje_Desempeno` sea igual a **4 o 5**.
2.  `Promedio_Dias_Ausente` sea **menor a 1**.
* *Pregunta 1:* ¿Cuántos empleados cumplen esto? ¿Quiénes son?

**B. Auditoría de "Sueldo vs. Rendimiento" (La Incoherencia)**
Queremos ver quiénes ganan mucho pero rinden poco.
1.  Ordena la tabla por `Salario_Mensual` de mayor a menor.
2.  Filtra aquellos que tengan un `Puntaje_Desempeno` de **1 o 2**.
* *Pregunta 2:* ¿Existe algún empleado con salario alto (ej. > $2000) que tenga bajo desempeño?

**C. Análisis Departamental (Carga de Trabajo)**
1.  Agrupa los datos por `Departamento`.
2.  Calcula el promedio de `Horas_Extra` por departamento.
* *Pregunta 3:* ¿Qué departamento está haciendo más horas extra?